In [1]:
import tensorboard
import tensorflow as tf
import keras 
import tensorflow_hub as hub
import tensorflow_text
import pandas as pd

In [2]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt


In [3]:
data = pd.read_csv('./Data/Dynasent/train.csv')

In [17]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
transfomed_label = encoder.fit_transform(data['gold_label'])
data['gold_label'] = transfomed_label

In [5]:
# data preprocessing
data['sentence'] = data['sentence'].apply(lambda x: x.lower())
data['sentence'] = data['sentence'].apply(lambda x: x.replace('\n', ' '))
data['sentence'] = data['sentence'].apply(lambda x: x.replace('\t', ' '))
data['sentence'] = data['sentence'].apply(lambda x: x.replace('\r', ' '))
data['sentence'] = data['sentence'].apply(lambda x: x.replace('\xa0', ' '))
data['sentence'] = data['sentence'].apply(lambda x: x.replace('\u200b', ' '))
data['sentence'] = data['sentence'].apply(lambda x: x.replace('\ufeff', ' '))

In [6]:
train_ds = tf.data.Dataset.from_tensor_slices((data['sentence'], data['gold_label']))

In [7]:
val_ds = pd.read_csv('./Data/Dynasent/valid.csv')

In [8]:
val_ds['sentence'] = val_ds['sentence'].apply(lambda x: x.lower())
val_ds['sentence'] = val_ds['sentence'].apply(lambda x: x.replace('\n', ' '))
val_ds['sentence'] = val_ds['sentence'].apply(lambda x: x.replace('\t', ' '))
val_ds['sentence'] = val_ds['sentence'].apply(lambda x: x.replace('\r', ' '))
val_ds['sentence'] = val_ds['sentence'].apply(lambda x: x.replace('\xa0', ' '))
val_ds['sentence'] = val_ds['sentence'].apply(lambda x: x.replace('\u200b', ' '))
val_ds['sentence'] = val_ds['sentence'].apply(lambda x: x.replace('\ufeff', ' '))

In [9]:
val_ds_label = encoder.transform(val_ds['gold_label'])
val_ds['gold_label'] = val_ds_label

val_ds = tf.data.Dataset.from_tensor_slices((val_ds['sentence'], val_ds['gold_label']))

In [10]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

train_ds = train_ds.shuffle(buffer_size=len(data)).batch(batch_size)
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = val_ds.shuffle(buffer_size=len(val_ds)).batch(batch_size)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [11]:
tfhub_encoder = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
tfhub_prepocess = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"


In [12]:
# create the deep learning model to classify text into three categories
text_input = keras.Input(shape=(), dtype='string', name='text_input')
preprocessing_layer = hub.KerasLayer(tfhub_prepocess, name='Preprocessing_for_BERT')
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer(tfhub_encoder, trainable=False, name='BERT_encoder')
outputs = encoder(encoder_inputs)
x = keras.layers.Dropout(0.2)(outputs['sequence_output'])
x = keras.layers.Dense(100, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(100, activation='relu')(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(50, activation='relu')(x)
x = keras.layers.Dropout(0.35)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(3, activation='softmax')(x)
model = keras.Model(inputs=[text_input], outputs=x)
model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',             
                metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text_input (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 Preprocessing_for_BERT (KerasL  {'input_mask': (Non  0          ['text_input[0][0]']             
 ayer)                          e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [13]:
epochs = 50
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=epochs,
                    verbose=1,
                    callbacks=[
                        keras.callbacks.TensorBoard(log_dir='./logs',
                                                    histogram_freq=1,
                                                    write_graph=True,
                                                    write_images=True),
                        keras.callbacks.EarlyStopping(patience=5,
                                                      monitor='val_loss')

                    ])


Epoch 1/50
2516/2516 [==============================] - 1448s 571ms/step - loss: 0.3556 - accuracy: 0.8459 - val_loss: 0.5002 - val_accuracy: 0.7486
Epoch 2/50
2516/2516 [==============================] - 1434s 570ms/step - loss: 0.3256 - accuracy: 0.8596 - val_loss: 0.4858 - val_accuracy: 0.7550
Epoch 3/50
2516/2516 [==============================] - 1432s 569ms/step - loss: 0.3126 - accuracy: 0.8656 - val_loss: 0.5070 - val_accuracy: 0.7439
Epoch 4/50
2516/2516 [==============================] - 1433s 569ms/step - loss: 0.3016 - accuracy: 0.8713 - val_loss: 0.4660 - val_accuracy: 0.7656
Epoch 5/50
2516/2516 [==============================] - 1443s 573ms/step - loss: 0.2912 - accuracy: 0.8764 - val_loss: 0.5068 - val_accuracy: 0.7503
Epoch 6/50
2516/2516 [==============================] - 1424s 566ms/step - loss: 0.2803 - accuracy: 0.8800 - val_loss: 0.4954 - val_accuracy: 0.7547
Epoch 7/50
2516/2516 [==============================] - 5499s 2s/step - loss: 0.2722 - accuracy: 0.8820 - 

In [22]:
# load test data
test_ds = pd.read_csv('./Data/Dynasent/test.csv')


test_ds['sentence'] = test_ds['sentence'].apply(lambda x: x.lower())
test_ds['sentence'] = test_ds['sentence'].apply(lambda x: x.replace('\n', ' '))
test_ds['sentence'] = test_ds['sentence'].apply(lambda x: x.replace('\t', ' '))
test_ds['sentence'] = test_ds['sentence'].apply(lambda x: x.replace('\r', ' '))
test_ds['sentence'] = test_ds['sentence'].apply(lambda x: x.replace('\xa0', ' '))
test_ds['sentence'] = test_ds['sentence'].apply(lambda x: x.replace('\u200b', ' '))
test_ds['sentence'] = test_ds['sentence'].apply(lambda x: x.replace('\ufeff', ' '))

In [23]:
# transform the test data using the same encoder
test_ds_label = encoder.transform(test_ds['gold_label'])
test_ds['gold_label'] = test_ds_label

# create the test dataset
test_ds = tf.data.Dataset.from_tensor_slices((test_ds['sentence'], test_ds['gold_label']))

In [24]:
# create the test dataset
test_ds = test_ds.shuffle(buffer_size=len(test_ds)).batch(batch_size)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


# evaluate the model
results = model.evaluate(test_ds, verbose=1, callbacks=[
    keras.callbacks.TensorBoard(log_dir='./logs',
                                histogram_freq=1,
                                write_graph=True,
                                write_images=True)]
)


# print the results
print('Test loss:', results[0])
print('Test accuracy:', results[1])

113/113 [==============================] - 47s 418ms/step - loss: 0.3371 - accuracy: 0.8906
Test loss: 0.3371028006076813
Test accuracy: 0.8905555605888367


In [26]:
#save the model
model.save('E:\SLIIT\Year 4\Research\Models\MindsetEvaluation\model.h5')